# Why I like Julia
by Victor Boussange

## Julia is expressive and understandable


- This allows you to easily contribute to packages ➡️ this makes you more useful in this world



- The language is **emoji friendly**! This is how you would define a Lotka Volterra system in Julia

In [1]:
function lotka_volterra!(du, u, p, t)
    α, β, γ, δ = p
    🐰, 🦊 = u
    d🐰, d🦊 = du
    d🐰 = α * 🐰 - β * 🦊 * 🐰
    d🦊 = γ*🐰 * 🦊  - δ * 🦊
end

lotka_volterra! (generic function with 1 method)

## Julia is interactive

- Amazing IDE with VS code and inline prompts

![](https://code.visualstudio.com/assets/docs/languages/julia/overview.png)

- Julia code can be **developed interactively** on a remote server. More on that later on.


## Community well organized

Documentation is consistent and pleasant to access

![](img/Optimization.jl.png)

## Package management

Julia provides a **built-in package** manager called `Pkg` for **managing packages and environments**. Users can create a new environment and add specific packages to it, and each environment has its own set of dependencies. Julia also allows users to switch between different environments easily.

<div align="center"><img src="https://pkgdocs.julialang.org/v1/assets/logo.png" width="400"></img> </div>

More on that later on!

## Productivity

- Research script can be easily transformed into **packages**, directly available to the community

## Execution speed!

During my PhD, I have been interested in eco-evolutionary dynamics

![](img/differentiation.png)


I needed to build an individual based model and run **MANY** simulations.

### [EvoId.jl](https://github.com/vboussange/EvoId.jl)
Evolutionary Individual based modelling, mathematically grounded. A user friendly package aimed at simulating the evolutionary dynamics of a population structured over a complex spatio-evolutionary structures.

<div align="center"><img src="https://github.com/vboussange/EvoId.jl/raw/master/docs/src/assets/gif_evoid.gif" width="400"></img> </div>

That got me myy first article!

![](img/combio.png)

## World class differential equation solver

`DifferentialEquations.jl` is one of the best library that exists for solving differential equations.

Differential equations are everywhere in science!

Here is a benchmark where the task consists of solving a certain differential equation model

![](https://docs.sciml.ai/SciMLBenchmarksOutput/stable/MultiLanguage/figures/wrapper_packages_2_1.png)

#### Resources
- [Julia micro-benchmarks](https://julialang.org/benchmarks/)
- [SciML benchmarks](https://docs.sciml.ai/SciMLBenchmarksOutput/stable/MultiLanguage/ode_wrapper_packages/)

I needed to play with many ecological (ODE) models. To facilitate this task, I have written

### [ParametricModels.jl](https://github.com/vboussange/ParametricModels.jl)
**A wrapper package to facilitate the handling of ODE models.**

and further used ParametricModels.jl to build

### [EcoEvoModelZoo.jl](https://github.com/vboussange/EcoEvoModelZoo.jl)
**A zoo of eco-evolutionary models with high fitness.**
<div align="center"><img src="https://github.com/vboussange/EcoEvoModelZoo.jl/raw/main/docs/src/time_series_5_species_ecosyste_model.png" width="800"></img> </div>

## Composability of libraries and scientific ML

### The [SciML](https://sciml.ai) ecosystem

SciML is a composable open source software for scientific machine learning with differentiable programming.
   -  Read [this very cool paper](https://arxiv.org/abs/2001.04385) introducing the [SciML software ecosystem]
 


  
#### [Using Deep Learning libraries in combination with ODE solvers](ode_solvers_with_DL.jl)

```julia
function lotka_volterra(u, p, t)
    weights_nn, α, δ = p
    🐰, 🦊 = u

    nn_res = neuralnet(🦊, 🐰, weights_nn)

    d🐰 = α * 🐰 - nn_res[1]
    d🦊 = nn_res[2]  - δ * 🦊

    return [d🐰, d🦊]
end
```

### Bayesian inference framework with deep learning

[Here is another cool example](https://turinglang.org/v0.24/tutorials/03-bayesian-neural-network/) of composability between [`Turing.jl`](https://turinglang.org/v0.24/) and `Flux.jl`

![](https://turinglang.org/v0.24/tutorials/figures/03_bayesian-neural-network_9_1.png)

It turns out I have contributed to SciML by releasing my first registered Julia package

### [HighDimPDE.jl](https://github.com/SciML/HighDimPDE.jl)
Solver for **highly dimensional, non-local, nonlinear PDEs** of the form

$$
\begin{aligned}
    (\partial_t u)(t,x) &= \int_{\Omega} f\big(t,x,{\bf x}, u(t,x),u(t,{\bf x}), ( \nabla_x u )(t,x ),( \nabla_x u )(t,{\bf x} ) \big) d{\bf x} \\
    & \quad + \big\langle \mu(t,x), ( \nabla_x u )( t,x ) \big\rangle + \tfrac{1}{2} \text{Trace} \big(\sigma(t,x) [ \sigma(t,x) ]^* ( \text{Hess}_x u)(t, x ) \big).
\end{aligned}
$$

where $u \colon [0,T] \times \Omega \to \mathbb{R}, \Omega \subseteq \mathbb{R}^{d}$ is subject to initial and boundary conditions, and where $d$ is large.

I have also relied on the SciML ecosystem to build a library purposed to perform inverse ecological modeling

### [PiecewiseInference.jl](https://github.com/vboussange/PiecewiseInference.jl)
Suite for parameter inference and model selection with dynamical models characterised by complex dynamics.
<div align="center"><img src="https://github.com/vboussange/PiecewiseInference.jl/raw/main/docs/animated.gif
" width="400"></img> </div>

## Caveat: trial-and-error development process

Julia is still in a trial and development process. You may expect some bugs in the libraries from time to time, or some rapid changes in syntax.